# Charity Through Marginal Price Discrimination

In this project we are going to explore and break down a very interesting post by Vitalik Buterin on his website:

https://vitalik.ca/jekyll/general/2017/03/11/a_note_on_charity.html

He believes that using marginal price discrimination, people can be incentivized (through social and economic pressures) to participate towards a certain cause. This is facilitated quite easily through the use of blockchain technology, where we can build an ecosystem of "stickers" (non-transferable digital tokens that organizations hand out to people who they think are contributing towards a worthy cause). 

Furthermore, being or working for a merchant that offers lower prices to sticker holders is enough to merit you one of these stickers. Buterin believes that by doing this it will make the scheme recursive. If an entire community approves of a particular cause, it will be profit-maximizing to start offering discounts for the associated sticker, causing everyone to maintain stable equilibrium in the level of spending and participation.

To explore further, click on the link above on how Vitalik Buterin plans to implement this scheme. However, he also provides a Python script that shows an economic simulation described in the post, and we will break it down here.

Buterin uses the example of sandwich buyers and sellers in the economic simulation. First assumption is that the cost of producing a sandwich is $6.

In [1]:
sandwich_production_cost = 6

Now we will create a list of agents, who each have their own reserve price for a sandwhich, and a certain utility (in dollars, including any warm feelings gained from doing something altruistic) from volunteering in a charity. We will do this by creating a Python list with tuples inside, each tuple representing one agent.

In [3]:
agents = []

for i in range(301): 
    for j in range(301):
        agents.append((6.0001 + i * 0.01, -4.501 + j * 0.02))

agents_sorted_by_charity_utility = sorted(agents, key = lambda f:f[1])

Let's break this down. 

First, we create a huge list of agents, in this case, since we create a for-loop within a for-loop that iterates 301 times each, we create a total of: 

301 x 301 = 90601 agents

Next, we see that for sandwich reserve price, it is evenly distributed between $6 - $9. To test it out put the first and last values of the iteration into the formula (note: index always starts at 0)

- At i = 0, 
sandwich_reserve_price = 6.0001 + 0 * 0.01 = 6.0001
- At i = 300, 
sandwich_reserve_price = 6.0001 + 300 * 0.01 = 9.0001

Similarly, the charity utility is evenly distributed between -$4.5 - $1.5, because some people might actually have negative utility from having to volunteer in a charity (without incentive or reward)

- At j = 0, 
charity_utility = -4.501 + 0 * 0.02 = -4.501
- At j = 300,
charity_utility = -4.501 + 300 * 0.02 = 1.499

Lastly, we sort the agents by charity utility, as shown in the "key = lambda f:f[1]" paramater in sorted(), because charity_utility is in index 1 (second position) in each tuple.

We can see also see that we have created a total of 90,601 agents

In [4]:
len(agents_sorted_by_charity_utility)

90601

Now, we will create a function that outputs the amount of profit the merchant gets and the number of charity volunteers, given the parameters: 

- Price of sandwich for charity volunteers
- Price of sandwich for non-volunteers 

In [5]:
def profit_and_charity_goers(sandwich_no_charity, sandwich_charity):
    
    # create variables to store totals of results 
    
    charity_goers = 0
    sandwich_buyers = 0 #both volunteers and non-volunteers
    revenue = 0
    
    # Iterate through each agent in the "agents" list
    
    for (sandwich_reserve_price, charity_utility) in agents:
        # Utility from doing neither buying a sandwich nor volunteering
        u_do_nothing = 0
        
        # Utility from buying a sandwhich (without volunteering)
        u_buy_sandwich = sandwich_reserve_price - sandwich_no_charity
        
        # Utility from going to charity (only)
        u_charity = charity_utility
        
        # Utility from buying sandwich + charity
        u_charity_plus_sandwich = charity_utility + sandwich_reserve_price - sandwich_charity
        
        # Now we find the max utility. If any utility within the ones we computed 
        # equals the max utility, means the action is taken 
        u_max = max(u_do_nothing, u_buy_sandwich, u_charity, u_charity_plus_sandwich)
        
        # In the iteration, does a particular agent volunteer at a charity?
        if u_max in(u_charity, u_charity_plus_sandwich):
            charity_goers += 1
        
        # In the iteration, does a particular agent buy a sandwich?
        if u_max in(u_buy_sandwich, u_charity_plus_sandwich):
            sandwich_buyers += 1
            # Calculate revenue of merchant gained from selling sandwich
            if u_charity_plus_sandwich == u_max:
                revenue += sandwich_charity
            else:
                revenue += sandwich_no_charity
        
    # Profit = Revenue - Cost 
    profit = revenue - (sandwich_buyers * sandwich_production_cost)
    
    # Return profit for merchants & number of charity goers
    return profit, charity_goers

Now that we have made our function, we are ready to make comparisons between the normal, non-price-discriminating case of selling sandwiches vs. various degrees of discounts given to charity volunteers in the simulation (Note that our function will output a tuple).

In this example, we will start with the original sandwich price of $7.5 and vary the price by:

- Increase the price of each sandwich by a multiple of 2 cents for non-volunteers
- Decrease the price of each sandwich by a multiple of 6 cents for charity volunteers

In [16]:
# Start at the original price of $7.5 to calculate 
# original total profit and volunteers
profit1, volunteers1 = profit_and_charity_goers(7.5, 7.5)

# Loop six times (0-6) with different variations as stated above
for i in range(6):
    profit2, volunteers2 = profit_and_charity_goers(7.5 + i * 0.02, 7.5 - i * 0.06)
    # Print results (floating point rounded to 2 decimals)
    print('At prices ($ %.2f, $ %.2f)' % (7.5 + i * 0.02, 7.5 - i * 0.06))
    print('Net cost to merchant: $ %.2f' % (profit1 - profit2))
    print('Net gain in charity goers: %.0f volunteers' % (volunteers2 - volunteers1))
    
    # Equivalent subsidy size to get equal charity volunteers (explained more below)
    equiv_subsidy_size = equiv_subsidy_size = agents_sorted_by_charity_utility[-volunteers2][1] - \
                         agents_sorted_by_charity_utility[-volunteers1][1]
    
    # Note that this subsidy would go to charity workers only
    print('Cost of providing equipotent straight subsidy: $ %.2f'
          % (equiv_subsidy_size * volunteers2))

At prices ($ 7.50, $ 7.50)
Net cost to merchant: $ 0.00
Net gain in charity goers: 0 volunteers
Cost of providing equipotent straight subsidy: $ 0.00
At prices ($ 7.52, $ 7.44)
Net cost to merchant: $ 60.66
Net gain in charity goers: 612 volunteers
Cost of providing equipotent straight subsidy: $ -1391.22
At prices ($ 7.54, $ 7.38)
Net cost to merchant: $ 243.96
Net gain in charity goers: 1240 volunteers
Cost of providing equipotent straight subsidy: $ -2381.50
At prices ($ 7.56, $ 7.32)
Net cost to merchant: $ 551.82
Net gain in charity goers: 1884 volunteers
Cost of providing equipotent straight subsidy: $ -3424.26
At prices ($ 7.58, $ 7.26)
Net cost to merchant: $ 986.16
Net gain in charity goers: 2544 volunteers
Cost of providing equipotent straight subsidy: $ -4521.42
At prices ($ 7.60, $ 7.20)
Net cost to merchant: $ 1548.90
Net gain in charity goers: 3220 volunteers
Cost of providing equipotent straight subsidy: $ -5674.90


First we will go through one of these printed statements. 

At a price of 7.52 for non-volunteers and 7.44 for volunteers, the merchant loses only 60.66 in profit, but gives an incentive that is enough to add an additional 612 volunteers within 90,601 agents.

Without this scheme, it will cost about 1,391.22 of government subsidy to provide the same incentive. Note that the list of agents are sorted form the lowest to highest charity utility. Therefore, we get this number by performing a few steps given in line 14 of the code above: 

1. Get the highest charity utility of the new number of volunteers (in dollars)

2. Get the highest charity utility of the original number of volunteers (in dollars)

3. Subtract them to get the "cost" of incentivizing more people to volunteer by only using government subsidy

Lastly, since the incentive will only go to charity workers (and not spread out through the equilibrium like in the simulation), we multiply the incentive we get from the calculation in line 14 by the new number of volunteers (given in line 18).

©Regio Abundan. May 29th, 2018.